In [1]:
import joblib
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# 1. Cargar los datos que guardamos antes
data_path = r"D:\UTP\Empresa Aeronáutica\aerosafe-risk-predictor\data\processed\model_data.pkl"
X_train, X_test, y_train, y_test = joblib.load(data_path)

# 2. Configurar el modelo
# Usamos class_weight='balanced' porque hay pocos casos de daño comparado con los de no daño
model_rf = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)

# 3. Entrenar el modelo (esto puede tardar unos segundos)
print("Entrenando el modelo Random Forest...")
model_rf.fit(X_train, y_train)

# 4. Realizar predicciones
y_pred = model_rf.predict(X_test)

# 5. Ver resultados
print("\n=== Reporte de Clasificación ===")
print(classification_report(y_test, y_pred))

print("\nROC-AUC Score:", roc_auc_score(y_test, model_rf.predict_proba(X_test)[:, 1]))

Entrenando el modelo Random Forest...

=== Reporte de Clasificación ===
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      4595
           1       0.43      0.25      0.31       491

    accuracy                           0.90      5086
   macro avg       0.68      0.61      0.63      5086
weighted avg       0.88      0.90      0.88      5086


ROC-AUC Score: 0.7728401764957483


In [2]:
import joblib
import os

models_dir = r"D:\UTP\Empresa Aeronáutica\aerosafe-risk-predictor\data\processed"
os.makedirs(models_dir, exist_ok=True)

joblib.dump(model_rf, os.path.join(models_dir, "rf_damage_model.pkl"))
print("Modelo Random Forest guardado en data/processed/rf_damage_model.pkl")

Modelo Random Forest guardado en data/processed/rf_damage_model.pkl


In [3]:
import pandas as pd
import joblib
import numpy as np

# Cargar preprocesador y modelo
processed_dir = r"D:\UTP\Empresa Aeronáutica\aerosafe-risk-predictor\data\processed"
preprocessor = joblib.load(os.path.join(processed_dir, "preprocessor_ohe.pkl"))
model_rf = joblib.load(os.path.join(processed_dir, "rf_damage_model.pkl"))

def predecir_daño(registro_dict):
    """
    registro_dict: diccionario con las mismas claves que X:
        AircraftType, AirportName, AltitudeBin, MakeModel,
        WildlifeSize, ConditionsSky, ConditionsPrecipitation
    """
    df_nuevo = pd.DataFrame([registro_dict])
    X_enc = preprocessor.transform(df_nuevo)
    prob_daño = model_rf.predict_proba(X_enc)[0, 1]
    pred = int(prob_daño >= 0.5)
    return pred, prob_daño

# Ejemplo de uso
ejemplo = {
    "AircraftType": "Airplane",
    "AirportName": "DALLAS/FORT WORTH INTL ARPT",
    "AltitudeBin": "(1000, 2000]",
    "MakeModel": "B-737-400",
    "WildlifeSize": "Medium",
    "ConditionsSky": "No Cloud",
    "ConditionsPrecipitation": "Unknown"
}

pred, prob = predecir_daño(ejemplo)
print("Predicción (1=Caused damage, 0=No damage):", pred)
print("Probabilidad de daño:", prob)

Predicción (1=Caused damage, 0=No damage): 0
Probabilidad de daño: 0.1777330015797832
